<a href="https://colab.research.google.com/github/droyktton/clases_ME_IB/blob/main/ME_2025_PseudoF_CampoMedio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis numérico de la pseudo energía libre y sus mínimos estables y metastables

Aquí van algunas sugerencias de como hacerlo.

Como ejemplo nos concentraremos en la pseudo energía de campo medio del Ising con $z$ vecinos (pueden descomentar/comentar las lineas adecuadas para probar la pseudo energía de Bargg Williams del mismo modelo).

$$\tilde{f}(\langle \sigma \rangle,T) = J\langle \sigma \rangle^2 \frac{z}{2} - \beta^{-1}\ln[2\cosh[\beta (Jz\langle \sigma \rangle+B)]$$

pero se puede hacer con cualquier pseudo energía libre.
En todos los casos $Jz=1$.

In [ ]:
#@title En python

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact


# Define the function f(x, y)
def f(M, T, h):
    # Bragg Williams era:
    #return -M**2/2 - h*M + T*((1 + M) * np.log((1 + M)/2)/2 + (1 - M) * np.log((1 - M)/2)/2)

    # Campo medio es:
    return M**2/2 - T*np.log(2*np.cosh((M+h)/T))

# Define the function f'(x, y)
def f1(M, T, h):
    #return -h - M +T*(-np.log((1-M)/2)+np.log((1+M)/2))/2
    return M - np.tanh((M+h)/T)

# Define the function f''(x, y)
def f2(M, T, h):
    return - 1 + T*(1/(1-M)+1/(1+M))/2

def plot_free_energy(t,h):
  m = np.linspace(-0.999 , 0.999, 400)
  y = f(m,t,h)
  plt.plot(m, y, label=r"$F(m,t)$")
  plt.show()

def plot_contour(h):
  # Create a grid of (x, y) points
  t = np.linspace(0.0001, 2, 400)
  m = np.linspace(-0.999 , 0.999, 400)
  M, T = np.meshgrid(m, t)

  # Evaluate the function on the grid
  Z = f1(M, T,h)

  # second derivative
  Z2 = f2(M, T,h)

  stable = Z2 > 0
  unstable = Z2 < 0


  filtered_Z = np.where(stable,Z,np.nan)
  plt.contour(M, T, filtered_Z, levels=[0], colors='blue')

  filtered_Z = np.where(unstable,Z,np.nan)
  plt.contour(M, T, filtered_Z, levels=[0], colors='red',linestyles='--')

  #print(stable.shape)

  # Customize plot
  plt.title(r"Contorno de $df/dm =  0$")
  plt.xlabel("M")
  plt.ylabel("T")
  plt.grid(True)
  plt.show()


interact(plot_contour, h=(-2.0, 2.0, 0.001))
interact(plot_free_energy,h=(-0.5, 0.5, 0.001),t=(0.001, 3.0, 0.1))



## En mathematica

Ver [ejemplo](https://www.wolframcloud.com/obj/9ab5c2e0-e54c-4376-bed8-cfa06dcb7997).

In [ ]:
#@title  En geogebra

# En gris la energía libre extendida, en marron su derivada primera, y en verde la derivada segunda.

%%html

<iframe src="https://www.geogebra.org/classic/xcbet6dx?embed" width="1000" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

## En gnuplot


In [ ]:
pip install gnuplot_kernel

In [ ]:
!sudo apt-get install gnuplot

In [ ]:
%load_ext gnuplot_kernel

In [ ]:
# prompt: contour plot in gnuplot only level 0 unset grid

%%gnuplot
res


minx=-1
maxx=1
miny=0
maxy=2

set view map;
unset surf;
set isosam 200,200
set cntrparam levels discrete 0.0;
set samples 200,200
set cont;
set xrange [minx:maxx]
set yrange [miny:maxy]
set xlabel 'm'
set ylabel 'T'
unset key


# energia libre de campo medio y sus derivadas
f(x,T,h)=-x**2/2 - h*x + T*((1 + x)*log((1 + x)/2)/2 + (1 - x)*log((1 - x)/2)/2)
f1(x,T,h)=-x - h + T*(log((1 + x)/2)/2 -log((1 - x)/2)/2)
f2(x,T,h)=-1 + T*(1/(2*(1 + x)) + 1/(2*(1 - x)))

# Set the number of samples (higher numbers give smoother contours)
#set samples 100, 100               # Number of sample points along x-axis
#set isosamples 100

#set multi lay 2,1
splot [minx:maxx][miny:maxy] (f2(x,y,0)>0)?(f1(x,y,0)):(1./0) lw 2 lc 1 t 'estable', \
 (f2(x,y,0)<0)?(f1(x,y,0)):(1./0) lw 2 lc 2 t 'inestable'

h=0.01
splot [minx:maxx][miny:maxy] (f2(x,y,h)>0)?(f1(x,y,h)):(1./0) lw 2 lc 1 t 'estable', \
 (f2(x,y,h)<0)?(f1(x,y,h)):(1./0) lw 2 lc 2 t 'inestable'
#unset multi